In [1]:
from dewloosh.solid.fem import FemMesh, TET4, Structure
from dewloosh.math.linalg import normalize
import pyvista as pv
import numpy as np

In [2]:
# geometry [cm]
d = 100.0
h = 1.0
a = 1.5
b = 0.5

# mesh options
regular_mesh = True
n_radii, n_angles, n_z = 20, 100, 40

# mass of the ball [kg]
m = 10
g = 9.81

# material
E = 12000
nu = 0.2

In [3]:
if regular_mesh:
    from dewloosh.geom.tri.trimesh import circular_disk
    from dewloosh.geom.utils import detach_mesh_bulk
    from dewloosh.geom.tet.tetutils import extrude_T3_TET4
    min_radius = b/2
    max_radius = a/2
    points, triangles = \
        circular_disk(n_angles, n_radii, min_radius, max_radius)
    points, triangles = detach_mesh_bulk(points, triangles)
    coords, topo = extrude_T3_TET4(points, triangles, h, n_z)
else:
    import tetgen
    cyl = pv.CylinderStructured(center=(0.0, 0.0, h/2), direction=(0.0, 0.0, 1.0),
                                radius=np.linspace(b/2, a/2, n_radii), height=h, 
                                theta_resolution=n_angles, z_resolution=n_z)
    cyl_surf = cyl.extract_surface().triangulate()
    tet = tetgen.TetGen(cyl_surf)
    tet.tetrahedralize(order=1, mindihedral=10, minratio=1.1, quality=True)
    grid = tet.grid
    coords = np.array(grid.points).astype(float)
    topo = grid.cells_dict[10].astype(np.int32)

In [4]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 2]) <= 1e-5
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 3), dtype=bool)
fixity[ebcinds, :] = True

In [5]:
# Hooke model
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E /(1-nu**2))

In [6]:
# natural boundary conditions
cond_nbc_1 = coords[:, 2] >= (h - 1e-8)
cond_nbc_2 = np.abs((coords[:, 0]**2 + coords[:, 1]**2)**(1/2) - b/2) <= 0.001
cond_nbc = cond_nbc_1 & cond_nbc_2
nbcinds = np.where(cond_nbc)[0]
q = (m * g * d) / (len(nbcinds) * np.sqrt(d**2-b**2))
c = np.array([0, 0, h + np.sqrt(d**2-b**2)/2])
foo = lambda x : normalize(x - c) * q
loads = np.zeros((coords.shape[0], 3))
loads[nbcinds] = [foo(coords[nID]) for nID in nbcinds]

In [7]:
mesh = FemMesh(coords=coords, topo=topo, celltype=TET4, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [8]:
mesh.plot()

In [11]:
structure.linsolve()
structure.summary

PyPardisoError: The Pardiso solver failed with error code -3. See Pardiso documentation for details.

In [26]:
dofsol = structure.mesh.pointdata.dofsol.to_numpy()
dofsol[:, 2].min()

-0.03509608398273206

In [27]:
from dewloosh.solid.topopt.SIMP_OC_FEM import OC_SIMP_COMP as OC

In [28]:
# iteration parameters
OC_params = {
    'p_start' : 1.0,  # SIMP penalty factor
    'p_stop' : 3.0,
    'p_inc' : 0.1,
    'p_step' : 5,
    'q' : 0.5,  # smoothing factor
    'vfrac' : 0.3,  # fraction of target volume over initial volume
    'dtol' : 0.1,  # to control maximum change in the variables
    'r_min' : (a-b)/2/10,  # for the density filter
    'miniter' : 150,
    'maxiter' : 300
    }
oc = OC(structure,  summary=True, **OC_params)

In [29]:
optres = next(oc)

In [30]:
try:
    for i in range(300):
        optres = next(oc)
        print("Iteration   {} :    vol {}     obj {}     pen {}".\
            format(optres.n, optres.vol, optres.obj, optres.pen))
except StopIteration:
    print("Iteration   {} :    vol {}     cobj {}     pen {}".\
            format(optres.n, optres.vol, optres.obj, optres.pen))
finally:
    print("TOTAL NUMBER OF ITERATIONS : {}".format(optres.n))

Iteration   0 :    vol 0.47093459640075047     obj 31.089544438911105     pen 1.0
Iteration   1 :    vol 0.4709344258916675     obj 26.31151351371163     pen 1.1
Iteration   2 :    vol 0.47094745794898624     obj 22.388174260172164     pen 1.1
Iteration   3 :    vol 0.47092152348202865     obj 19.16458399004137     pen 1.1
Iteration   4 :    vol 0.4709331811505234     obj 16.511763397513114     pen 1.1
Iteration   5 :    vol 0.4709302310856791     obj 14.32686787902686     pen 1.1
Iteration   6 :    vol 0.4709281433307578     obj 12.536413677278633     pen 1.2000000000000002
Iteration   7 :    vol 0.4709153112124423     obj 11.05491219424971     pen 1.2000000000000002
Iteration   8 :    vol 0.470924300734922     obj 9.830943944401813     pen 1.2000000000000002
Iteration   9 :    vol 0.47093259276202704     obj 8.81972565046517     pen 1.2000000000000002
Iteration   10 :    vol 0.4709240927498987     obj 7.983565950440104     pen 1.2000000000000002
Iteration   11 :    vol 0.470942205187

In [31]:
from dewloosh.geom import PolyData, Tetra

In [33]:
topo_opt = topo[np.where(optres.x >= 0.99)[0]]
pd = PolyData(coords=coords, topo=topo_opt, celltype=Tetra)
pd.plot(notebook=False)